#### Check code details of JTT

In [1]:
#packages
import os
import sys

import numpy as np
import pandas as pd

import torch
import torchvision

sys.path.append("..")
from data import get_data, get_loader, get_group_repr
from utils.data import UpsampleSampler



#settings
res_dir = "/scratch/ym2380/data/celebA/results"

ex_fname = "11291120_5"
ex_conditional_epoch = 8
ex_up_weight = 20

loader_kwargs = {
    "batch_size": 64,
    "num_workers": 1,
    "pin_memory": True,
}

class FakeArgs:
    method = "JTT"
    dataset = "celebA"
    train_fraction = 1
    model = "resnet50"
    root = "/scratch/ym2380/data/"
    exp_name = ex_fname
    conditional_epoch = ex_conditional_epoch
    up_weight = ex_up_weight
args = FakeArgs()



#data
train_set = get_data(args, "train")
train_loader = get_loader(dataset=train_set, train=True, args=args, **loader_kwargs)

DEBUG PURPOSE len of the sampler is: 398959


In [2]:
#orig stat
def display_orig_stat(args=args, res_dir=res_dir):
    fdf = pd.read_csv(os.path.join(res_dir, args.exp_name))
    
    res_name = f"e{args.conditional_epoch:02d}"
    target_name = "Blond_Hair"
    mask = fdf[res_name] != fdf[target_name]
    upsample_indices = np.where(mask)[0]
    
    print(f"Orig Data Stat:\nTotal: {fdf.shape[0]}\nPart2: {len(upsample_indices)}\nRatio: {len(upsample_indices) / fdf.shape[0] * 100:.3f}%\n")
    return upsample_indices
    


#new stat
def display_new_stat1(upsample_indices, dataset=train_set, args=args):
    upsample_sampler = UpsampleSampler(
        upsample_indices = upsample_indices,
        n_samples=len(dataset),
        up_weight=args.up_weight,
    )
    print(len(upsample_sampler))
    
def display_new_stat2(dataloader):
    num_data_points = 0
    for batch_idx, batch in enumerate(dataloader):
        batch = tuple(e for e in batch)
        X, y, g, data_idx = batch[0], batch[1], batch[2], batch[3]
        num_data_points += len(X)
    print(f"New Data Stat:\nTotal: {num_data_points}\n")

In [3]:
upsample_indices = display_orig_stat()

Orig Data Stat:
Total: 162770
Part2: 12431
Ratio: 7.637%



In [4]:
display_new_stat2(train_loader)

New Data Stat:
Total: 398959



In [5]:
display_new_stat1(upsample_indices)

398959


In [6]:
162770 - 12431 + 12431 * 20

398959